# Setup

In [1]:
import dataloader
import torch
import numpy as np
import argparse

In [2]:
# Arguments ===================================
parser = argparse.ArgumentParser()

parser.add_argument('--gpu_id', type=int, default=1)
parser.add_argument('--seed', type=int, default=123)
parser.add_argument('--config', type=str, default='config.yaml')
parser.add_argument('--dataset', type=str, default='cora', choices=['cora', 'citeseer', 'BlogCatalog', 'flickr', 'Polblogs'])
parser.add_argument('--g0_method', type=str, default='many_clusters', choices=[
  'random', # randomly distribution of g0
  'bias', # a random class has a 3x higher likelihood of being in g0
  'large_cluster', # a random node and [g0_size] of its neighbors are in g0
  'many_clusters', # 10 random nodes and [g0_size] of their neighbors are in g0
  ])
parser.add_argument('--g0_size', type=float, default=0.1)
parser.add_argument('--attack_method', type=str, default='sll', choices=[
  'sll', # Selective Learnability Lock
  'sll_no_g', # Disable gradient guidance
  'noise', # Noise protection
  'heuristic' # Heuristic protection
  ])
parser.add_argument('--budget_pct', type=float, default=0.25)
parser.add_argument('--attack_epochs', type=int, default=30)
parser.add_argument('--save_results', type=str, default='Y', choices=['N', 'Y'])
parser.add_argument('--save_graph', type=str, default='N', choices=['N', 'Y'])

args = parser.parse_args("")

if args.gpu_id >= 0:
  device = torch.device(f'cuda:{args.gpu_id}' if torch.cuda.is_available() else 'cpu')
else:
  device = torch.device('cpu')

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if device != 'cpu': torch.cuda.manual_seed(args.seed)

import yaml
from yaml import SafeLoader
config = yaml.load(open('config.yml'), Loader=SafeLoader)[args.dataset]

In [3]:
# Load graph ===================================
graph = dataloader.load_DGL(args.dataset)
feat = graph.ndata['feat'].to(device)
labels = graph.ndata['label'].to(device)
adj = graph.adj().to_dense()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [26]:
import models

comparisonMLP = models.SimpleMLP(
  in_size=feat.shape[1],
  out_size=labels.max().item()+1,
  hid_size=config['hid_size'],
  lr=config['lr'],
  dropout=config['dropout'],
  weight_decay=config['weight_decay']
).to(device)

comparisonMLP.fit(feat, labels, 100)
acc = (comparisonMLP(feat).argmax(dim=1) == labels).sum() / feat.shape[0]
print(f"Accuracy: {acc:.2%}")

GCN Training: 100%|██████████| 100/100 [00:00<00:00, 332.56it/s, loss=0.86]

Accuracy: 78.47%


In [18]:
def compareModels(feat: torch.Tensor, adj: torch.Tensor, task_idx):
    featClone = feat.clone()
    featClone[task_idx]

tensor([2492,  786, 1520, 2125, 2598,  291, 2018], device='cuda:1')

In [4]:
# Designate g0 ===================================
g0_size = int(args.g0_size * graph.num_nodes())

def get_clusters(num_roots: int, max_hops: int, target_size: int) -> torch.tensor:
  root_nodes = torch.rand(graph.num_nodes()).topk(num_roots).indices

  for hop in range(max_hops):
    newNodes = adj[root_nodes].nonzero().t()[1]
    root_nodes = torch.cat((root_nodes, newNodes))
    root_nodes = torch.unique(root_nodes)
    if root_nodes.shape[0] >= target_size:
      break

  g0 = torch.zeros(graph.num_nodes())
  g0[root_nodes[:target_size]] = 1
  g0 = g0.bool()
  return g0

if args.g0_method == 'many_clusters': # 10 nodes and their neighbors
  g0 = get_clusters(10, 10, g0_size)
elif args.g0_method == 'large_cluster': # 1 node and its neighbors
  g0 = get_clusters(1, 10, g0_size)
elif args.g0_method == 'random': # g0 is random/bias
  g0_probs = torch.ones(graph.num_nodes())
  g0_probs = g0_probs * (g0_size / g0_probs.sum())
  g0_probs.clamp_(0, 1)
  g0 = torch.bernoulli(g0_probs).bool()
elif args.g0_method == 'bias': # g0 is skewed toward a class by factor of 3
  bias = torch.randint(0, labels.max() + 1, [1]).item()
  print(f'G0 class bias: {bias}')
  g0_probs = torch.ones(graph.num_nodes())
  g0_probs[labels == bias] = 3
  g0_probs = g0_probs * (g0_size / g0_probs.sum())
  g0_probs.clamp_(0, 1)
  g0 = torch.bernoulli(g0_probs).bool()

print(f'G0 size: {g0.sum().item()}')
print(f'G0 pct: {g0.sum().item() / graph.num_nodes():.2%}')

g0 = g0.cpu()
gX = ~g0

G0 size: 270
G0 pct: 9.97%


# Method

In [5]:
import importlib
import sampling_matrix
importlib.reload(sampling_matrix)

<module 'sampling_matrix' from '/u/nyw6dh/HCDM/Experiment/LLFeat/sampling_matrix.py'>

In [143]:
class SamplingMatrix:
  def __init__(self, g0, gX, sample_size):
    self.sample_size = sample_size
    self.g0_idx = g0.nonzero().squeeze()
    self.gX_idx = gX.nonzero().squeeze()
    self.balance = torch.tensor([1, 1, 1]) # g0, gX, g0gX

    self.normalize_balance()
  
  def normalize_balance(self):
    self.balance = self.balance / self.balance.sum()

  def update_balance(self, new_ratio):
    self.balance = (self.balance + new_ratio) / 2
    self.normalize_balance()

  def _sample(self, sample_size, edge_idx):
    idx_sample = torch.tensor(random.choices(range(edge_idx.shape[0]), k=sample_size))
    return edge_idx[idx_sample]
  
  def _sample_edges(self, sample_size, start_idx, end_idx):
    sample_size = int(sample_size)
    return torch.stack((self._sample(sample_size, start_idx), self._sample(sample_size, end_idx)))

  def sample(self):
    g0_samples = self._sample_edges(self.sample_size * self.balance[0], self.g0_idx, self.g0_idx)
    gX_samples = self._sample_edges(self.sample_size * self.balance[1], self.gX_idx, self.gX_idx)
    g0gX_samples = self._sample_edges(self.sample_size * self.balance[2], self.g0_idx, self.gX_idx)

    return torch.cat((g0_samples, gX_samples, g0gX_samples), dim=1)

a = SamplingMatrix(g0, gX, 2000)
sample = a.sample()

import scipy
reverse = scipy.stats.rankdata(sample, method='dense', axis=1) - 1

adj[sample]

torch.Size([2, 1998, 2708])

In [147]:
sample, reverse.max()

(tensor([[1546, 2377, 1527,  ..., 1212, 1963, 2600],
         [2616, 2244, 1816,  ...,  935,  715,  439]]),
 1287)

In [145]:
flatten = torch.zeros(adj.shape[0], dtype=torch.bool)
flatten[torch.flatten(sample)] = True

sampled_adj = adj[flatten, :][:, flatten].to(device)
sampled_adj.shape

torch.Size([1665, 1665])

In [88]:
config['sll_sample_size']

2000

In [83]:
# Perform attack ==========================

import utils
import models
import torch.nn.functional as F
from tqdm import tqdm

num_perturbations = (graph.num_edges() / 2) * args.budget_pct

print(f'Attacking with method: {args.attack_method}')

if args.attack_method == 'heuristic':
  locked_adj = adj.clone()
  locked_adj[:, g0] = 0
  locked_adj[g0, :] = 0
elif args.attack_method == 'noise':
  noise = torch.zeros_like(adj)
  noise[g0, :] = 1
  noise[:, gX] = 0
  noise *= 2 * num_perturbations / noise.sum()
  noise = torch.bernoulli(noise.clamp(0, 1))
  noise = utils.make_symmetric(noise)
  locked_adj = utils.get_modified_adj(adj, noise)
elif args.attack_method == 'sll_no_g':

  # Initialize perturbations
  perturbations = torch.zeros_like(adj).float()

  # Create surrogate model to mimic downstream
  surrogate = models.DenseGCN(
      in_size=feat.shape[1],
      out_size=labels.max().item()+1,
      hid_size=config['hid_size'],
      lr=config['lr'],
      dropout=config['dropout'],
      weight_decay=config['weight_decay']
  ).to(device)

  t = tqdm(range(args.attack_epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
  t.set_description("SLL (no gradient guidance)")

  for epoch in t:
    # Re-initialize adj_grad
    adj_grad = torch.zeros_like(adj).float()

    # Get modified adj
    modified_adj = utils.get_modified_adj(adj, perturbations).requires_grad_(True).float().to(device)

    # Get grad of modified adj w.r.t attack loss
    pred = surrogate(feat, modified_adj)
    loss = F.cross_entropy(pred[g0], labels[g0]) \
        - F.cross_entropy(pred[gX], labels[gX])
    adj_grad = torch.autograd.grad(loss, modified_adj)[0].cpu()

    # Update perturbations
    lr = (config['sll_no_g_lr']) / ((epoch + 1))
    pre_projection = int(perturbations.sum() / 2)
    perturbations = perturbations + (lr * adj_grad)
    perturbations = utils.projection(perturbations, num_perturbations)

    # Train the surrogate
    modified_adj = utils.get_modified_adj(adj, perturbations).to(device)
    model_loss = surrogate.fit(feat, modified_adj, labels, epochs=1, verbose=False)

    t.set_postfix({"adj_l": loss.item(),
                    "adj_g": (adj_grad.sum().item()),
                    "pre-p": pre_projection,
                    "target": int(num_perturbations / 2),
                    "model_loss": model_loss})
elif args.attack_method == 'sll':
  # Initialize perturbations
  perturbations = torch.zeros_like(adj).float()

  # Initialize sampling matrix
  samplingMatrix = sampling_matrix.SamplingMatrix(
    g0=g0, gX=gX, sample_size=config['sll_sample_size'])
  count = torch.zeros_like(adj).float()

  # Create surrogate model to mimic downstream
  surrogate = models.DenseGCN(
      in_size=feat.shape[1],
      out_size=labels.max().item()+1,
      hid_size=config['hid_size'],
      lr=config['lr'],
      dropout=config['dropout'],
      weight_decay=config['weight_decay']
  ).to(device)

  t = tqdm(range(args.attack_epochs), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
  t.set_description("SLL")

  for epoch in t:
    # Re-initialize adj_grad
    adj_grad = torch.zeros_like(adj).float()

    # Get modified adj
    modified_adj = utils.get_modified_adj(adj, perturbations).float()

    for sample_epoch in range(config['sll_num_samples']): 
      # Get sample indices
      idx = samplingMatrix.get_sample_pairs()
      rev = idx.argsort()

      flatten = torch.zeros(modified_adj.shape[0], dtype=torch.bool)
      flatten[torch.flatten(idx)] = True

      sampled_adj = modified_adj[flatten, :][:, flatten].to(device)
      sampled_feat = feat[flatten].to(device)
      
      rev_cuda = rev.to(device)
      idx_cuda = idx.to(device)

      sample = sampled_adj[rev[0], rev[1]].clone().detach().requires_grad_(True).to(device)
      sampled_adj[rev[0], rev[1]] = sample

      # Get grad
      pred = surrogate(sampled_feat, sampled_adj)
      loss = F.cross_entropy(pred[g0[flatten]], labels[flatten][g0[flatten]]) \
          - F.cross_entropy(pred[gX[flatten]], labels[flatten][gX[flatten]])

      grad = torch.autograd.grad(loss, sample)[0].cpu()

      # Implement averaging of duplicate samples
      adj_grad[idx[0], idx[1]] += grad
      count[idx[0], idx[1]] += 1


    # Update the sampling matrix
    samplingMatrix.updateByGrad(adj_grad, count)

    # Average the gradient
    adj_grad = torch.div(adj_grad, count)
    adj_grad[adj_grad != adj_grad] = 0
    
    # Update perturbations
    lr = (config['sll_lr']) / ((epoch + 1))
    pre_projection = int(perturbations.sum())
    perturbations = perturbations + (lr * adj_grad)

    perturbations[graph.edges()[0], graph.edges()[1]].mul_(-1)
    perturbations.clamp_(0, 1)

    for k in range(5):
      perturbations = (perturbations * (num_perturbations / perturbations.sum())).clamp(-1, 1)
      if abs((perturbations.sum() / num_perturbations) - 1) > 0.9: break

    # perturbations = utils.projection(perturbations, num_perturbations)

    # Train the model
    modified_adj = utils.get_modified_adj(adj, perturbations).to(device)
    model_loss = surrogate.fit(feat, modified_adj, labels, epochs=1, verbose=False)

    t.set_postfix({"attack_loss": loss.item(),
                    # "adj_g": (adj_grad.sum().item()),
                    "pre-projection": pre_projection,
                    "target": int(num_perturbations),
                    "surrogate_loss": model_loss})

# Discretize the best locked_adj ============
if args.attack_method in ['sll', 'sll_no_g']:
  locked_adj = None
  with torch.no_grad():
    max_loss = -1000
    for k in range(0,3):
        sample = torch.bernoulli(perturbations)
        modified_adj = utils.get_modified_adj(adj, sample)
        modified_adj = utils.make_symmetric(modified_adj) 
        predictions = surrogate(feat, modified_adj.to(device)) 

        loss = F.cross_entropy(predictions[g0], labels[g0]) \
            - F.cross_entropy(predictions[gX], labels[gX])

        if loss > max_loss:
            max_loss = loss
            locked_adj = modified_adj
    
    print(f"Best sample loss: {max_loss:.2f}")

diff = adj - locked_adj
print(f'Edges modified: {diff.abs().sum() / 2:.0f}')

Attacking with method: sll


SLL:   0%|          | 0/30 [00:00<?, ?it/s]


AttributeError: 'SamplingMatrix' object has no attribute 'get_sample_pairs'

In [14]:
# Evaluation ==============================
import sklearn.metrics as metrics
gX_train = torch.logical_and(gX, graph.ndata['train_mask'])
gX_test = torch.logical_and(gX, graph.ndata['test_mask'])

def eval_adj(test_adj: torch.tensor):
    model = models.DenseGCN(
        in_size=feat.shape[1],
        out_size=labels.max().item()+1,
        hid_size=config['hid_size'],
        lr=config['lr'],
        dropout=config['dropout'],
        weight_decay=config['weight_decay']
    ).to(device)
    model.fit(feat, test_adj, labels, epochs=100, mask=gX_train)
    pred = model(feat, test_adj).cpu()

    f1_g0 = metrics.f1_score(labels[g0].cpu(), pred.argmax(dim=1)[g0], average='micro')
    f1_gX = metrics.f1_score(labels[gX_test].cpu(), pred.argmax(dim=1)[gX_test], average='micro')
    
    return f1_g0, f1_gX

f1_g0_base, f1_gX_base = eval_adj(adj)
f1_g0_lock, f1_gX_lock = eval_adj(locked_adj)

d_g0 = (f1_g0_lock - f1_g0_base) / f1_g0_base
d_gX = (f1_gX_lock - f1_gX_base) / f1_gX_base


GCN Training: 100%|██████████| 100/100 [00:01<00:00, 56.40it/s, loss=0.46]


In [15]:
import importlib
importlib.reload(utils)

H_overall = utils.calc_homophily(adj, labels)

H_g0_base = utils.calc_homophily(adj, labels, g0)
H_gX_base = utils.calc_homophily(adj, labels, gX)
H_g0gX_base = utils.inner_homophily(adj, labels, g0, gX)

H_g0_lock = utils.calc_homophily(locked_adj, labels, g0)
H_gX_lock = utils.calc_homophily(locked_adj, labels, gX)
H_g0gX_lock = utils.inner_homophily(locked_adj, labels, g0, gX)

d_H_g0 = (H_g0_lock - H_g0_base) / H_g0_base
d_H_gX = (H_gX_lock - H_gX_base) / H_gX_base
d_H_g0gX = (H_g0gX_lock - H_g0gX_base) / H_g0gX_base

changes_g0 = diff[g0, :][:, g0].abs().sum().item()
changes_gX = diff[gX, :][:, gX].abs().sum().item()
changes_g0gX = diff.abs().sum().item() - (changes_g0 + changes_gX)

print(f'H Overall: {H_overall:.1%}')
print(f'ACC      f1_g0\tf1_gX\tH_g0\tH_gX\tH_g0gX')
print(f'base   | {f1_g0_base:.1%}\t{f1_gX_base:.1%}\t{H_g0_base:.1%}\t{H_gX_base:.1%}\t{H_g0gX_base:.1%}')
print(f'lock   | {f1_g0_lock:.1%}\t{f1_gX_lock:.1%}\t{H_g0_lock:.1%}\t{H_gX_lock:.1%}\t{H_g0gX_lock:.1%}')
print(f'delta  | {d_g0:.1%}\t{d_gX:.1%}\t{d_H_g0:.1%}\t{d_H_gX:.1%}\t{d_H_g0gX:.1%}')

print(f'Changes')
print(f'g0: {changes_g0}')
print(f'gX: {changes_gX}')
print(f'g0gX: {changes_g0gX}')

H Overall: 81.0%
ACC      f1_g0	f1_gX	H_g0	H_gX	H_g0gX
base   | 74.0%	74.1%	84.6%	81.7%	81.0%
lock   | 52.6%	71.2%	19.6%	79.9%	68.3%
delta  | -28.9%	-4.0%	-76.8%	-2.2%	-15.7%
Changes
g0: 640.0
gX: 394.0
g0gX: 1528.0
